In [130]:
import pandas as pd
import numpy as np

In [131]:
df = pd.read_csv('/Users/13630/Downloads/stancedetection-main/dataset_stance.csv')

In [132]:
import sklearn

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [134]:
import nltk

In [135]:
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Error loading averaged_perceptron_tagger: HTTP Error 401:
[nltk_data]     Unauthorized
[nltk_data] Error loading stopwords: HTTP Error 401: Unauthorized


False

In [136]:
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Error loading averaged_perceptron_tagger: HTTP Error 401:
[nltk_data]     Unauthorized


False

In [137]:
# WORDNET LEMMATIZER (with appropriate pos tags)

from nltk.stem import WordNetLemmatizer
#nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Error loading stopwords: HTTP Error 401: Unauthorized


False

In [138]:
# WORDNET LEMMATIZER (with appropriate pos tags)


lemmatizer = WordNetLemmatizer()
 
# Define function to lemmatize each word with its POS tag
 
# POS_TAGGER_FUNCTION : TYPE 1
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None
 
def lemmatizing(data):
    lemmatized_data = []
    for i in range(len(data)):
        sentence = data['Tweet'][i]

        # tokenize the sentence and find the POS tag for each token
        word_tokens = nltk.wordpunct_tokenize(sentence) #what does wordpunct_tokenize do?
        pos_tagged = nltk.pos_tag(word_tokens) 

        
        # As you may have noticed, the above pos tags are a little confusing.
        
        # we use our own pos_tagger function to make things simpler to understand.
        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
        
        lemmatized_sentence = []
        for word, tag in wordnet_tagged:
            word = word.lower()
            if word.isdigit():
                del word
            elif tag is None:
                # if there is no available tag, append the token as is
                lemmatized_sentence.append(word)
            else:       
                # else use the tag to lemmatize the token
                lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
        lemmatized_sentence = " ".join(lemmatized_sentence)
        lemmatized_data.append(lemmatized_sentence)
    return lemmatized_data

In [139]:
#no separation based on subtopic
def grouping(df):
    df_for = []
    df_against = []
    df_neutral = []
    for k in range(len(df)):
        if df['Stance'][k] == 'FAVOR':
            df_for.append(df['Tweet'][k]) 
        elif df['Stance'][k] == 'AGAINST':
            df_against.append(df['Tweet'][k])
        else:
            df_neutral.append(df['Tweet'][k])
    return df_for, df_against, df_neutral

In [140]:
grouped_df = df.groupby(['Target'])
df_atheism = grouped_df.get_group('Atheism')
df_climate = grouped_df.get_group('Climate Change is a Real Concern')
df_trump = grouped_df.get_group('Donald Trump')
df_feminist = grouped_df.get_group('Feminist Movement')
df_hillary = grouped_df.get_group('Hillary Clinton')
df_abortion = grouped_df.get_group('Legalization of Abortion')

consider how to count punctuation too. are they considered by the vectorizer function?

In [141]:
#tfidfVectorizer
def tfidf_vectorizer(list):
    vectorizer = TfidfVectorizer(ngram_range=(1,3))
    matrix = vectorizer.fit_transform(list)
    idf_table = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names_out())
    return idf_table

# Donald Trump 

In [142]:
df_trump

,Unnamed: 0,Tweet,Target,Stance,Opinion Towards,Sentiment
4163,1249,@2014voteblue @ChrisJZullo blindly supporting ...,Donald Trump,NONE,2. The tweet does NOT expresses opinion about ...,neg
4164,1250,@ThePimpernelX @Cameron_Gray @CalebHowe Total...,Donald Trump,NONE,2. The tweet does NOT expresses opinion about ...,pos
4165,1251,@JeffYoung @ThePatriot143 I fully support full...,Donald Trump,NONE,2. The tweet does NOT expresses opinion about ...,pos
4166,1252,@ABC Stupid is as stupid does! Showedhis true ...,Donald Trump,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4167,1253,@HouseGOP we now have one political party. The...,Donald Trump,NONE,2. The tweet does NOT expresses opinion about ...,neg
...,...,...,...,...,...,...
4865,1951,@realDonaldTrump we all want you as the next p...,Donald Trump,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
4866,1952,@RSherman_25 Join Twitter Trump brigade #oneth...,Donald Trump,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
4867,1953,@JoeyBats19 Join Twitter Trump brigade #onetho...,Donald Trump,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
4868,1954,Trump's outlandish statements is political str...,Donald Trump,AGAINST,1. The tweet explicitly expresses opinion abo...,neg


In [143]:
df_trump = df_trump.drop('Unnamed: 0', axis='columns')
df_trump = df_trump.reset_index()
df_trump = df_trump.drop('index', axis = 'columns')

df_trump

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,@2014voteblue @ChrisJZullo blindly supporting ...,Donald Trump,NONE,2. The tweet does NOT expresses opinion about ...,neg
1,@ThePimpernelX @Cameron_Gray @CalebHowe Total...,Donald Trump,NONE,2. The tweet does NOT expresses opinion about ...,pos
2,@JeffYoung @ThePatriot143 I fully support full...,Donald Trump,NONE,2. The tweet does NOT expresses opinion about ...,pos
3,@ABC Stupid is as stupid does! Showedhis true ...,Donald Trump,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,@HouseGOP we now have one political party. The...,Donald Trump,NONE,2. The tweet does NOT expresses opinion about ...,neg
...,...,...,...,...,...
702,@realDonaldTrump we all want you as the next p...,Donald Trump,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
703,@RSherman_25 Join Twitter Trump brigade #oneth...,Donald Trump,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
704,@JoeyBats19 Join Twitter Trump brigade #onetho...,Donald Trump,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
705,Trump's outlandish statements is political str...,Donald Trump,AGAINST,1. The tweet explicitly expresses opinion abo...,neg


In [144]:
lemmatized_tweets = lemmatizing(df_trump)

In [145]:
table = tfidf_vectorizer(lemmatized_tweets)
table

,10thamendment,10thamendment funny,10thamendment funny visit,1130aet,1130aet semst,1962kiser,1962kiser fanofgreenmms,1962kiser fanofgreenmms ericbolling,1a,1a semst,...,yup,yup you,yup you get,zekejmiller,zekejmiller reince,zekejmiller reince sharonday,zero,zero semst,zero sense,zero sense nbc
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [146]:
#table = table.transpose()

In [147]:
# labels for donald trump df
list_of_labels = []
for t in range(len(df_trump)):
    if df_trump['Stance'][t] == 'FAVOR':
        list_of_labels.append(1)
    elif df_trump['Stance'][t] == 'AGAINST':
        list_of_labels.append(2)
    else:
        list_of_labels.append(0) 

In [148]:
list_of_labels

[0,
 0,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 1,
 2,
 1,
 1,
 2,
 1,
 0,
 0,
 2,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 0,
 2,
 0,
 2,
 2,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [149]:
X = tfidf_vectorizer(lemmatized_tweets)
y = list_of_labels

In [150]:
# apply naive bayes machine learning algorithm 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
from sklearn.naive_bayes import MultinomialNB
NB_classifier = MultinomialNB()
NB_classifier.fit(X_train, y_train)
MultinomialNB(alpha = 0.1, class_prior = None, fit_prior = True)

MultinomialNB(alpha=0.1)

In [151]:
# Obtain model's predictions 
from sklearn.metrics import classification_report, confusion_matrix
y_pred_test = NB_classifier.predict(X_test)
CM = print(confusion_matrix(y_test, y_pred_test))

[[ 7  0 45]
 [ 0  0 29]
 [ 5  0 56]]


In [152]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.58      0.13      0.22        52
           1       0.00      0.00      0.00        29
           2       0.43      0.92      0.59        61

    accuracy                           0.44       142
   macro avg       0.34      0.35      0.27       142
weighted avg       0.40      0.44      0.33       142



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Hillary Clinton 

In [153]:
df_hillary

,Unnamed: 0,Tweet,Target,Stance,Opinion Towards,Sentiment
0,0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
1,1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
2,2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
3,3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,3. The tweet is not explicitly expressing opi...,pos
...,...,...,...,...,...,...
3878,964,.@HillaryClinton Looking 4ward 2 hearing your ...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
3879,965,@mataharikrishna I'm loving it too! Draw that ...,Hillary Clinton,NONE,2. The tweet does NOT expresses opinion about ...,pos
3880,966,"@finneyk Can't stand @msnbc anymore, but hope ...",Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
3881,967,Hillary can't create jobs! Last time she had a...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg


In [154]:
df_hillary = df_hillary.reset_index()
df_hillary = df_hillary.drop('index', axis = 'columns')

df_hillary

,Unnamed: 0,Tweet,Target,Stance,Opinion Towards,Sentiment
0,0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
1,1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
2,2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
3,3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,3. The tweet is not explicitly expressing opi...,pos
...,...,...,...,...,...,...
979,964,.@HillaryClinton Looking 4ward 2 hearing your ...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
980,965,@mataharikrishna I'm loving it too! Draw that ...,Hillary Clinton,NONE,2. The tweet does NOT expresses opinion about ...,pos
981,966,"@finneyk Can't stand @msnbc anymore, but hope ...",Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
982,967,Hillary can't create jobs! Last time she had a...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg


In [208]:
lemmatized_tweets = lemmatizing(df_hillary)

In [209]:
table = tfidf_vectorizer(lemmatized_tweets)
table 

,100k,100k speech,100k speech to,100m,100m on,100m on direct,10am,10am pt,10am pt semst,11s,...,zaksteib great connection,zekejmiller,zekejmiller bretbaier,zekejmiller bretbaier danmericacnn,zerlinamaxwell,zerlinamaxwell huge,zerlinamaxwell huge cheer,zero,zero accomplishment,zero accomplishment hillary
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
982,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [210]:
#table = table.transpose()

In [211]:
# labels for hillary df
list_of_labels = []
for t in range(len(df_hillary)):
    if df_hillary['Stance'][t] == 'FAVOR':
        list_of_labels.append(1)
    elif df_hillary['Stance'][t] == 'AGAINST':
        list_of_labels.append(2)
    else:
        list_of_labels.append(0) 

In [212]:
list_of_labels

[2,
 1,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 1,
 0,
 2,
 0,
 1,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 0,
 1,
 2,
 1,
 2,
 0,
 1,
 0,
 2,
 1,
 2,
 0,
 1,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 1,
 1,
 1,
 0,
 0,
 1,
 2,
 0,
 2,
 2,
 1,
 2,
 0,
 1,
 2,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 1,
 2,
 2,
 0,
 1,
 1,
 2,
 2,
 2,
 1,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 1,
 0,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 0,
 0,
 1,
 2,
 2,
 1,
 0,
 2,
 1,
 2,
 0,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 2,
 0,
 2,
 1,
 1,
 2,
 0,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 0,
 2,
 1,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 1,
 2,


In [213]:
X = tfidf_vectorizer(lemmatized_tweets)
y = list_of_labels

In [214]:
# apply naive bayes machine learning algorithm 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
from sklearn.naive_bayes import MultinomialNB
NB_classifier = MultinomialNB()
NB_classifier.fit(X_train, y_train)
MultinomialNB(alpha = 0.1, class_prior = None, fit_prior = True)

MultinomialNB(alpha=0.1)

In [215]:
# Obtain model's predictions 
from sklearn.metrics import classification_report, confusion_matrix
y_pred_test = NB_classifier.predict(X_test)
CM = print(confusion_matrix(y_test, y_pred_test))

[[  1   0  46]
 [  0   1  32]
 [  0   0 117]]


In [216]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      0.02      0.04        47
           1       1.00      0.03      0.06        33
           2       0.60      1.00      0.75       117

    accuracy                           0.60       197
   macro avg       0.87      0.35      0.28       197
weighted avg       0.76      0.60      0.47       197



# Feminist Movement 

In [217]:
df_feminist

,Unnamed: 0,Tweet,Target,Stance,Opinion Towards,Sentiment
0,1008,Always a delight to see chest-drumming alpha m...,Feminist Movement,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
1,1009,Sometimes I overheat and want to take off my s...,Feminist Movement,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
2,1010,If feminists spent 1/2 as much time reading pa...,Feminist Movement,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
3,1011,"Stupid Feminists, the civilization you take fo...",Feminist Movement,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,1012,YOU'RE A GIRL AND HAVE A SEX DRIVE!? YOU MUST ...,Feminist Movement,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
...,...,...,...,...,...,...
944,669,@Maisie_Williams is our hero with her #LikeAGi...,Feminist Movement,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
945,670,"Rather be an ""ugly"" feminist then be these sad...",Feminist Movement,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
946,671,iamNovaah: RT ChrzOC: Bitches be running wild....,Feminist Movement,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
947,672,@angerelle you disagree that people should str...,Feminist Movement,FAVOR,2. The tweet does NOT expresses opinion about ...,neg


In [218]:
df_feminist = df_feminist.reset_index()
df_feminist = df_feminist.drop('index', axis = 'columns')

df_feminist

,Unnamed: 0,Tweet,Target,Stance,Opinion Towards,Sentiment
0,1008,Always a delight to see chest-drumming alpha m...,Feminist Movement,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
1,1009,Sometimes I overheat and want to take off my s...,Feminist Movement,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
2,1010,If feminists spent 1/2 as much time reading pa...,Feminist Movement,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
3,1011,"Stupid Feminists, the civilization you take fo...",Feminist Movement,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,1012,YOU'RE A GIRL AND HAVE A SEX DRIVE!? YOU MUST ...,Feminist Movement,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
...,...,...,...,...,...,...
944,669,@Maisie_Williams is our hero with her #LikeAGi...,Feminist Movement,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
945,670,"Rather be an ""ugly"" feminist then be these sad...",Feminist Movement,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
946,671,iamNovaah: RT ChrzOC: Bitches be running wild....,Feminist Movement,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
947,672,@angerelle you disagree that people should str...,Feminist Movement,FAVOR,2. The tweet does NOT expresses opinion about ...,neg


In [219]:
lemmatized_tweets = lemmatizing(df_feminist)

In [220]:
table = tfidf_vectorizer(lemmatized_tweets)
table 

,1960s,1960s semst,1hhhm18,1hhhm18 semst,1st,1st world,1st world be,20th,20th century,20th century semst,...,zero semst,zip,zip up,zip up and,zoe,zoe quinn,zoe quinn be,zombieneith,zombieneith feel,zombieneith feel decides
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [221]:
#table = table.transpose()

In [222]:
list_of_labels = []
for t in range(len(df_feminist)):
    if df_feminist['Stance'][t] == 'FAVOR':
        list_of_labels.append(1)
    elif df_feminist['Stance'][t] == 'AGAINST':
        list_of_labels.append(2)
    else:
        list_of_labels.append(0) 

In [223]:
list_of_labels

[1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 1,
 2,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 2,
 1,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 1,
 2,
 2,
 2,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 1,
 2,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 2,
 1,
 2,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 1,
 0,
 1,
 1,
 1,
 2,
 0,
 1,
 0,
 1,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 0,
 1,
 0,
 2,
 0,
 0,
 1,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 2,
 2,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 0,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 0,
 0,
 1,


In [224]:
X = tfidf_vectorizer(lemmatized_tweets)
y = list_of_labels

In [225]:
# apply naive bayes machine learning algorithm 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
from sklearn.naive_bayes import MultinomialNB
NB_classifier = MultinomialNB()
NB_classifier.fit(X_train, y_train)
MultinomialNB(alpha = 0.1, class_prior = None, fit_prior = True)

MultinomialNB(alpha=0.1)

In [226]:
# Obtain model's predictions 
from sklearn.metrics import classification_report, confusion_matrix
y_pred_test = NB_classifier.predict(X_test)
CM = print(confusion_matrix(y_test, y_pred_test))

[[  0   0  26]
 [  0   1  53]
 [  0   0 110]]


In [227]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        26
           1       1.00      0.02      0.04        54
           2       0.58      1.00      0.74       110

    accuracy                           0.58       190
   macro avg       0.53      0.34      0.26       190
weighted avg       0.62      0.58      0.44       190



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Legalization of Abortion

In [228]:
df_abortion

,index,Unnamed: 0,Tweet,Target,Stance,Opinion Towards,Sentiment
0,50,50,Just laid down the law on abortion in my bioet...,Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
1,51,51,@tooprettyclub Are you OK with #GOP males tell...,Legalization of Abortion,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
2,52,52,"If you don't want your kid, put it up for adop...",Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
3,53,53,"@RedAlert -there should be a ""stigma"" to butch...",Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,54,54,But isn't that the problem then. Not enough fa...,Legalization of Abortion,NONE,2. The tweet does NOT expresses opinion about ...,neg
...,...,...,...,...,...,...,...
928,4158,1244,@MetalheadMonty @tom_six I followed him before...,Legalization of Abortion,NONE,2. The tweet does NOT expresses opinion about ...,neg
929,4159,1245,"For he who avenges blood remembers, he does no...",Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,other
930,4160,1246,Life is sacred on all levels. Abortion does no...,Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,other
931,4161,1247,"@ravensymone U refer to ""WE"" which =""YOU"" & a ...",Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,neg


In [229]:
df_abortion = df_abortion.reset_index()
df_abortion

,level_0,index,Unnamed: 0,Tweet,Target,Stance,Opinion Towards,Sentiment
0,0,50,50,Just laid down the law on abortion in my bioet...,Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
1,1,51,51,@tooprettyclub Are you OK with #GOP males tell...,Legalization of Abortion,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
2,2,52,52,"If you don't want your kid, put it up for adop...",Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
3,3,53,53,"@RedAlert -there should be a ""stigma"" to butch...",Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,4,54,54,But isn't that the problem then. Not enough fa...,Legalization of Abortion,NONE,2. The tweet does NOT expresses opinion about ...,neg
...,...,...,...,...,...,...,...,...
928,928,4158,1244,@MetalheadMonty @tom_six I followed him before...,Legalization of Abortion,NONE,2. The tweet does NOT expresses opinion about ...,neg
929,929,4159,1245,"For he who avenges blood remembers, he does no...",Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,other
930,930,4160,1246,Life is sacred on all levels. Abortion does no...,Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,other
931,931,4161,1247,"@ravensymone U refer to ""WE"" which =""YOU"" & a ...",Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,neg


In [230]:
lemmatized_tweets = lemmatizing(df_abortion)

In [231]:
table = tfidf_vectorizer(lemmatized_tweets)
table

,12th,12th week,12th week or,1310edhand,1310edhand 1310news,1310edhand 1310news we,1310news,1310news we,1310news we do,15minutes,...,yr9 class,yr9 class morality,yup,yup one,yup one of,yup the,yup the woman,ze,ze momento,ze momento just
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [232]:
#table = table.transpose()

In [233]:
# labels for donald trump df
list_of_labels = []
for t in range(len(df_abortion)):
    if df_abortion['Stance'][t] == 'FAVOR':
        list_of_labels.append(1)
    elif df_abortion['Stance'][t] == 'AGAINST':
        list_of_labels.append(2)
    else:
        list_of_labels.append(0) 

In [234]:
list_of_labels

[2,
 1,
 2,
 2,
 0,
 2,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 2,
 0,
 0,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 0,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 0,
 2,
 2,
 1,
 0,
 0,
 2,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 1,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 1,
 0,
 2,
 1,
 0,


In [235]:
X = tfidf_vectorizer(lemmatized_tweets)
y = list_of_labels

In [236]:
# apply naive bayes machine learning algorithm 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
from sklearn.naive_bayes import MultinomialNB
NB_classifier = MultinomialNB()
NB_classifier.fit(X_train, y_train)
MultinomialNB(alpha = 0.1, class_prior = None, fit_prior = True)

MultinomialNB(alpha=0.1)

In [237]:
# Obtain model's predictions 
from sklearn.metrics import classification_report, confusion_matrix
y_pred_test = NB_classifier.predict(X_test)
CM = print(confusion_matrix(y_test, y_pred_test))

[[  0   0  49]
 [  0   0  34]
 [  0   0 104]]


In [238]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        49
           1       0.00      0.00      0.00        34
           2       0.56      1.00      0.71       104

    accuracy                           0.56       187
   macro avg       0.19      0.33      0.24       187
weighted avg       0.31      0.56      0.40       187



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Atheism

In [239]:
df_atheism

,index,Unnamed: 0,Tweet,Target,Stance,Opinion Towards,Sentiment
0,100,100,dear lord thank u for all of ur blessings forg...,Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,pos
1,101,101,"Blessed are the peacemakers, for they shall be...",Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,pos
2,102,102,I am not conformed to this world. I am transfo...,Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,pos
3,103,103,Salah should be prayed with #focus and #unders...,Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,pos
4,104,104,And stay in your houses and do not display you...,Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,neg
...,...,...,...,...,...,...,...
728,3129,215,1of 2 #Never be #afraid to and never #apologis...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos
729,3130,216,How soon do you think WWIII &WWWIV will begin?...,Atheism,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
730,3131,217,"The humble trust in God: 'Whoever leans on, tr...",Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos
731,3132,218,Where will you be when @NewHorizons2015 flies ...,Atheism,FAVOR,1. The tweet explicitly expresses opinion abo...,pos


In [240]:
df_atheism = df_atheism.reset_index()
df_atheism

,level_0,index,Unnamed: 0,Tweet,Target,Stance,Opinion Towards,Sentiment
0,0,100,100,dear lord thank u for all of ur blessings forg...,Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,pos
1,1,101,101,"Blessed are the peacemakers, for they shall be...",Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,pos
2,2,102,102,I am not conformed to this world. I am transfo...,Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,pos
3,3,103,103,Salah should be prayed with #focus and #unders...,Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,pos
4,4,104,104,And stay in your houses and do not display you...,Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,neg
...,...,...,...,...,...,...,...,...
728,728,3129,215,1of 2 #Never be #afraid to and never #apologis...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos
729,729,3130,216,How soon do you think WWIII &WWWIV will begin?...,Atheism,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
730,730,3131,217,"The humble trust in God: 'Whoever leans on, tr...",Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos
731,731,3132,218,Where will you be when @NewHorizons2015 flies ...,Atheism,FAVOR,1. The tweet explicitly expresses opinion abo...,pos


In [241]:
lemmatized_tweets = lemmatizing(df_atheism)

In [242]:
table = tfidf_vectorizer(lemmatized_tweets)
table

,100aeuassembly,100aeuassembly celebrate,100aeuassembly celebrate ethicalunion,100aeuassembly semst,100th,100th anniversary,100th anniversary amazing,10x,10x bad,10x bad bethankful,...,zealot from,zealot from take,zealot lovewins,zealot lovewins semst,zmanoj,zmanoj sanghparivarorg,zmanoj sanghparivarorg when,zubair,zubair ibn,zubair ibn awwam
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [243]:
#table = table.transpose()

In [244]:

list_of_labels = []
for t in range(len(df_atheism)):
    if df_atheism['Stance'][t] == 'FAVOR':
        list_of_labels.append(1)
    elif df_atheism['Stance'][t] == 'AGAINST':
        list_of_labels.append(2)
    else:
        list_of_labels.append(0) 

In [245]:
list_of_labels

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 1,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 0,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 0,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 0,
 1,
 1,
 0,
 2,
 2,
 1,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 2,
 2,
 0,
 2,
 1,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 1,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 1,
 1,
 2,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 2,
 0,
 0,
 2,
 0,
 0,
 1,
 1,


In [246]:
X = tfidf_vectorizer(lemmatized_tweets)
y = list_of_labels

In [247]:
# apply naive bayes machine learning algorithm 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
from sklearn.naive_bayes import MultinomialNB
NB_classifier = MultinomialNB()
NB_classifier.fit(X_train, y_train)
MultinomialNB(alpha = 0.1, class_prior = None, fit_prior = True)

MultinomialNB(alpha=0.1)

In [248]:
# Obtain model's predictions 
from sklearn.metrics import classification_report, confusion_matrix
y_pred_test = NB_classifier.predict(X_test)
CM = print(confusion_matrix(y_test, y_pred_test))

[[  0   0  23]
 [  0   0  22]
 [  0   0 102]]


In [249]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.00      0.00      0.00        22
           2       0.69      1.00      0.82       102

    accuracy                           0.69       147
   macro avg       0.23      0.33      0.27       147
weighted avg       0.48      0.69      0.57       147



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Climate Change is a Real Concern

In [250]:
df_climate

,index,Unnamed: 0,Tweet,Target,Stance,Opinion Towards,Sentiment
0,613,613,"We cant deny it, its really happening. #SemST",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,other
1,614,614,RT @cderworiz: Timelines are short. Strategy m...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
2,615,615,SO EXCITING! Meaningful climate change action ...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
3,616,616,"Delivering good jobs for Albertans, maintainin...",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
4,617,617,@davidswann says he wants carbon fund to be sp...,Climate Change is a Real Concern,FAVOR,3. The tweet is not explicitly expressing opi...,other
...,...,...,...,...,...,...,...
559,3298,384,"@LiberalAus - ""All"" you folks #owe me an #offi...",Climate Change is a Real Concern,NONE,3. The tweet is not explicitly expressing opi...,neg
560,3299,385,"Maybe we should start ""labelling every #produc...",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
561,3300,386,If we touch #Antarctica for one more #reason o...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
562,3301,387,Wind power helped stabilize Texas' power grid ...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos


In [251]:
df_climate = df_climate.reset_index()
df_climate

,level_0,index,Unnamed: 0,Tweet,Target,Stance,Opinion Towards,Sentiment
0,0,613,613,"We cant deny it, its really happening. #SemST",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,other
1,1,614,614,RT @cderworiz: Timelines are short. Strategy m...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
2,2,615,615,SO EXCITING! Meaningful climate change action ...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
3,3,616,616,"Delivering good jobs for Albertans, maintainin...",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
4,4,617,617,@davidswann says he wants carbon fund to be sp...,Climate Change is a Real Concern,FAVOR,3. The tweet is not explicitly expressing opi...,other
...,...,...,...,...,...,...,...,...
559,559,3298,384,"@LiberalAus - ""All"" you folks #owe me an #offi...",Climate Change is a Real Concern,NONE,3. The tweet is not explicitly expressing opi...,neg
560,560,3299,385,"Maybe we should start ""labelling every #produc...",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
561,561,3300,386,If we touch #Antarctica for one more #reason o...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
562,562,3301,387,Wind power helped stabilize Texas' power grid ...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos


In [252]:
lemmatized_tweets = lemmatizing(df_climate)

In [253]:
table = tfidf_vectorizer(lemmatized_tweets)
table

,100k,100k more,100k more follower,10a,10a floridaproblems,10a floridaproblems semst,10mm,10mm into,10mm into 1mmm,1g9uil5,...,zerofootprint rt,zerofootprint rt climatereality,zim,zim semst,zinta,zinta zommers,zinta zommers need,zommers,zommers need,zommers need for
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
560,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [254]:
#table = table.transpose()

In [255]:

list_of_labels = []
for t in range(len(df_climate)):
    if df_climate['Stance'][t] == 'FAVOR':
        list_of_labels.append(1)
    elif df_climate['Stance'][t] == 'AGAINST':
        list_of_labels.append(2)
    else:
        list_of_labels.append(0) 

In [256]:
list_of_labels

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,


In [257]:
X = tfidf_vectorizer(lemmatized_tweets)
y = list_of_labels

In [258]:
# apply naive bayes machine learning algorithm 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
from sklearn.naive_bayes import MultinomialNB
NB_classifier = MultinomialNB()
NB_classifier.fit(X_train, y_train)
MultinomialNB(alpha = 0.1, class_prior = None, fit_prior = True)

MultinomialNB(alpha=0.1)

In [259]:
# Obtain model's predictions 
from sklearn.metrics import classification_report, confusion_matrix
y_pred_test = NB_classifier.predict(X_test)
CM = print(confusion_matrix(y_test, y_pred_test))

[[ 2 35  0]
 [ 0 69  0]
 [ 0  7  0]]


In [260]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      0.05      0.10        37
           1       0.62      1.00      0.77        69
           2       0.00      0.00      0.00         7

    accuracy                           0.63       113
   macro avg       0.54      0.35      0.29       113
weighted avg       0.71      0.63      0.50       113



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
